In [1]:
import numpy as np
import pyclesperanto_prototype as cle
from skimage.data import cells3d
from skimage.segmentation import morphological_chan_vese
import timeit
import stackview
from skimage.io import imread
cle.select_device("TX")

<NVIDIA GeForce RTX 3050 Ti Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

In [2]:
def morphological_chan_vese_(image, num_iter, init_level_set='checkerboard',
                            smoothing=1, lambda1=1, lambda2=1,
                            iter_callback=lambda x: None):
    """Morphological Active Contours without Edges (MorphACWE)

    Active contours without edges implemented with morphological operators. It
    can be used to segment objects in images and volumes without well defined
    borders. It is required that the inside of the object looks different on
    average than the outside (i.e., the inner area of the object should be
    darker or lighter than the outer area on average).

    Parameters
    ----------
    image : (M, N) or (L, M, N) array
        Grayscale image or volume to be segmented.
    num_iter : uint
        Number of num_iter to run
    init_level_set : str, (M, N) array, or (L, M, N) array
        Initial level set. If an array is given, it will be binarized and used
        as the initial level set. If a string is given, it defines the method
        to generate a reasonable initial level set with the shape of the
        `image`. Accepted values are 'checkerboard' and 'disk'. See the
        documentation of `checkerboard_level_set` and `disk_level_set`
        respectively for details about how these level sets are created.
    smoothing : uint, optional
        Number of times the smoothing operator is applied per iteration.
        Reasonable values are around 1-4. Larger values lead to smoother
        segmentations.
    lambda1 : float, optional
        Weight parameter for the outer region. If `lambda1` is larger than
        `lambda2`, the outer region will contain a larger range of values than
        the inner region.
    lambda2 : float, optional
        Weight parameter for the inner region. If `lambda2` is larger than
        `lambda1`, the inner region will contain a larger range of values than
        the outer region.
    iter_callback : function, optional
        If given, this function is called once per iteration with the current
        level set as the only argument. This is useful for debugging or for
        plotting intermediate results during the evolution.

    Returns
    -------
    out : (M, N) or (L, M, N) array
        Final segmentation (i.e., the final level set)

    See Also
    --------
    disk_level_set, checkerboard_level_set

    Notes
    -----
    This is a version of the Chan-Vese algorithm that uses morphological
    operators instead of solving a partial differential equation (PDE) for the
    evolution of the contour. The set of morphological operators used in this
    algorithm are proved to be infinitesimally equivalent to the Chan-Vese PDE
    (see [1]_). However, morphological operators are do not suffer from the
    numerical stability issues typically found in PDEs (it is not necessary to
    find the right time step for the evolution), and are computationally
    faster.

    The algorithm and its theoretical derivation are described in [1]_.

    References
    ----------
    .. [1] A Morphological Approach to Curvature-based Evolution of Curves and
           Surfaces, Pablo Márquez-Neila, Luis Baumela, Luis Álvarez. In IEEE
           Transactions on Pattern Analysis and Machine Intelligence (PAMI),
           2014, :DOI:`10.1109/TPAMI.2013.106`
    """
    from skimage.segmentation.morphsnakes import _check_input, _curvop, _init_level_set

    init_level_set = _init_level_set(init_level_set, image.shape)

    _check_input(image, init_level_set)

    u = np.int8(init_level_set > 0)

    iter_callback(u)

    for _ in range(num_iter):

        # inside = u > 0
        # outside = u <= 0
        c0 = (image * (1 - u)).sum() / float((1 - u).sum() + 1e-8)
        c1 = (image * u).sum() / float(u.sum() + 1e-8)

        # Image attachment
        du = np.gradient(u)
        abs_du = np.abs(du).sum(0)
        aux = abs_du * (lambda1 * (image - c1)**2 - lambda2 * (image - c0)**2)

        u[aux < 0] = 1
        u[aux > 0] = 0
        
        from skimage.io import imsave
        imsave("image1.tif", u)

        # Smoothing
        for _ in range(smoothing):
            u = _curvop(u)

        iter_callback(u)

    return u

In [3]:
image = cells3d()[:,0,...]
#cle.asarray(image)

In [4]:
def level_set_gpu():
    result_gpu = cle.morphological_chan_vese(image, num_iter=50, smoothing=0)

def level_set_cpu():
    result_cpu = morphological_chan_vese(image, num_iter=50, smoothing=0)

In [5]:
#time_in_s = timeit.timeit(level_set_gpu, number=3)
#print("GPU time = ", time_in_s)

In [6]:
#time_in_s = timeit.timeit(level_set_cpu, number=3)
#print("CPU time = ", time_in_s)

In [7]:
image2d = cells3d()[30,0,...]

In [8]:
result_gpu = cle.morphological_chan_vese(image2d, num_iter=20, smoothing=10)
#result_gpu

In [9]:
result_cpu = morphological_chan_vese(image2d, num_iter=20, smoothing=10)
#stackview.insight(result_cpu)

In [10]:
stackview.curtain(result_cpu, np.asarray(result_gpu))

In [11]:
image1 = imread("../../test1.tif")
image2 = imread("../../test2.tif")

stackview.curtain(image1, image2)

In [12]:
cle.array_equal(image1, image2)

False

In [13]:
stackview.curtain(result_cpu, np.asarray(result_gpu))

In [14]:
cle.array_equal(result_cpu, result_gpu)

True

In [15]:
from skimage.data import cells3d
from skimage.segmentation import morphological_chan_vese
import pyclesperanto_prototype as cle

image2d = cells3d()[30, 0, ...]

result_gpu = cle.morphological_chan_vese(image2d, num_iter=20, smoothing=10)
result_cpu = morphological_chan_vese(image2d, num_iter=20, smoothing=10)

from skimage.io import imsave
imsave("test1.tif", result_cpu)
imsave("test2.tif", result_gpu)

cle.array_equal(result_cpu, result_gpu)

C:\Users\haase\AppData\Local\Temp\ipykernel_23936\336360938.py:11: UserWarning: test1.tif is a low contrast image
  imsave("test1.tif", result_cpu)


True

In [16]:
from skimage.data import cells3d
from skimage.segmentation import morphological_chan_vese
import pyclesperanto_prototype as cle

image3d = cells3d()[10:50:, 0, :40, :40]

result_gpu = cle.morphological_chan_vese(image3d, num_iter=20, smoothing=10)
result_cpu = morphological_chan_vese(image3d, num_iter=20, smoothing=10)

from skimage.io import imsave
imsave("test1.tif", result_cpu)
imsave("test2.tif", result_gpu)

cle.array_equal(result_cpu, result_gpu)

C:\Users\haase\AppData\Local\Temp\ipykernel_23936\1440285567.py:11: UserWarning: test1.tif is a low contrast image
  imsave("test1.tif", result_cpu)


True